In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
import math
from IPython.display import display, Image
import tiktoken
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

In [2]:
url = "https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F81c2aa73-dd8c-46bf-85b0-90e01145b0ed_1422x1460.png"
display(Image(url=url,
              width=400,
              height=500))

In [3]:
@dataclass
class Config:
    n_layers:int = 12
    d_model:int = 768
    eps:float = 1e-5
    hidden_size_multiplier:int = 4 
    num_heads:int = 12
    context_len:int = 1024
    dropout:float = 0.1
    qkv_bias:bool = False
    vocab_size:int = 50257
    
config = Config()
device = torch.device('mps' if torch.mps.is_available() else 'cpu')

In [4]:
tokenizer = tiktoken.get_encoding('gpt2')

In [5]:
# Layer Normalization layer
class LayerNormalization(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.eps = config.eps
        self.scale = nn.Parameter(torch.ones((config.d_model)))
        self.shift = nn.Parameter(torch.zeros((config.d_model))) 
        
    def forward(self, x: torch.Tensor):
        x_mean = x.mean(dim=-1, keepdim=True) 
        x_std = x.std(dim=-1, keepdim=True)
        x_norm = (x - x_mean) / (x_std + self.eps)
        return x * self.scale + self.shift

In [6]:
# FeedForward Layer
class FeedForwardLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        d_model = config.d_model
        hidden_size_multiplier = config.hidden_size_multiplier
        
        self.ff_layer = nn.Sequential(
            nn.Linear(d_model, hidden_size_multiplier * d_model),
            nn.GELU(),
            nn.Linear(hidden_size_multiplier * d_model, d_model)        
        )
        
    def forward(self, x):
        return x + self.ff_layer(x)

In [7]:
class SelfAttentionLayer(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.query_weights = nn.Linear(config.d_model, config.d_model, bias=config.qkv_bias)
        self.key_weights = nn.Linear(config.d_model, config.d_model, bias=config.qkv_bias)
        self.value_weights = nn.Linear(config.d_model, config.d_model, bias=config.qkv_bias)
        self.out_proj = nn.Linear(config.d_model, config.d_model)
        self.num_heads = config.num_heads
        assert config.d_model % config.num_heads == 0, "d_model should be divisible by num_heads"
        self.h_dmodel = config.d_model // config.num_heads
        self.neg_inf = - 1e+5
        self.drop_out = torch.nn.Dropout(config.dropout)
        self.register_buffer('casual_mask', tensor=torch.triu(torch.ones(config.context_len, config.context_len), diagonal=1).bool())
    
    def forward(self, x):
        # x: (B, S, d_model)
        qeury_vectors = self.query_weights(x)
        key_vectors = self.key_weights(x)
        value_vectors = self.value_weights(x)
        batch_size, seq_len, d_model = x.size()
        
        # (B,S,d_model) -> (B, S, num_head, h_dmodel)
        qeury_vectors = qeury_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        key_vectors = key_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        value_vectors = value_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        
        # (B, Seq, num_heads, h_dmodel) -> (B, num_heads, Seq, h_dmodel)
        qeury_vectors = torch.permute(qeury_vectors, dims=(0, 2, 1, 3))
        key_vectors = torch.permute(key_vectors, dims=(0, 2, 1, 3))
        value_vectors = torch.permute(value_vectors, dims=(0, 2, 1, 3))
        mask = self.casual_mask[ :seq_len, : seq_len]
        
        # mask = self.casual_mask[:seq_len, :seq_len]  # (S, S)
        # mask = mask.unsqueeze(0).unsqueeze(0)  # (1, 1, S, S)
         
        attention_scores = self.calculate_attention_score(qeury_vectors, key_vectors, mask)
        contextualized_vectores = attention_scores @ value_vectors
        
        # (B, num_heads, seq, head_d) => (B, seq, num_head, head_d) => (b, seq, d_model)
        contextualized_vectores = torch.permute(contextualized_vectores, dims=(0, 2, 1, 3))
        contextualized_vectores = contextualized_vectores.contiguous().view(batch_size, seq_len, self.num_heads*self.h_dmodel)
        contextualized_vectores = self.out_proj(contextualized_vectores)
        return (contextualized_vectores, attention_scores)
    
    def calculate_attention_score(self, qeury, key, mask):
        # (B,NumHeads,Seq, h_dmodel) * (B,num_heads,h_model, seq) => (B,num_heads, seq, seq)
        k_dmodel = key.size(-1)
        attention_scores = (qeury @ key.transpose(-1,-2)) / math.sqrt(k_dmodel)
        attention_scores = torch.masked_fill(attention_scores, mask=mask, value=self.neg_inf)
        attention_scores = torch.softmax(attention_scores, dim=-1)
        return self.drop_out(attention_scores)

In [8]:
4*3*256

3072

In [9]:
class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, max_len, stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        tokens = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
        
        for i in range(0, len(tokens) - max_len, stride):
            self.input_ids.append(torch.tensor(tokens[i: i + max_len]))
            self.target_ids.append(torch.tensor(tokens[i+1 : i + max_len+1]))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [10]:
def create_dataloader(txt, batch_size=4, max_len=256,
                      stride=256, shuffle=True,
                      drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDataset(text=txt, tokenizer=tokenizer, max_len=max_len, stride=stride)
    
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=True,
        drop_last=True,
        num_workers=num_workers
    )
    return dataloader

In [11]:
class TransformerBlock(nn.Module):
    def __init__(self,config: Config):
        super().__init__()
        self.attention = SelfAttentionLayer(config)
        self.layer_norm1 = LayerNormalization(config)
        self.layer_norm2 = LayerNormalization(config)
        self.feedforward = FeedForwardLayer(config)
        self.dropout = nn.Dropout(config.dropout)
        
    def forward(self, x):
        shortcut = x # Residual connection
        x = self.layer_norm1(x)
        x, _ = self.attention(x)
        x = self.dropout(x)
        x = shortcut + x
        
        shortcut = x
        x = self.layer_norm2(x)
        x = self.feedforward(x)
        x = self.dropout(x)
        x = x + shortcut
        return x

In [12]:
class GPTModel(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.token_embedding = nn.Embedding(config.vocab_size, embedding_dim=config.d_model)
        self.pos_embedding = nn.Embedding(config.context_len, embedding_dim=config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        
        self.decoder_block = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config.n_layers)]
            )
        
        self.final_layernorm = LayerNormalization(config)
        self.final_linear = nn.Linear(config.d_model, config.vocab_size)
        
    def forward(self, x):
        batch_size, seq_len = x.size()
        token_embedding = self.token_embedding(x)
        positions = torch.arange(0, seq_len, device=x.device)
        pos_embedding = self.pos_embedding(positions)
        x = token_embedding + pos_embedding
        x = self.decoder_block(x)
        x = self.dropout(x)
        x = self.final_layernorm(x)
        logits = self.final_linear(x)
        return logits

In [13]:
# model = GPTModel(config=config)

In [14]:
# model.token_embedding.weight

In [15]:
# torch.manual_seed(10)
# x = torch.randint(high=10,size=(1,5),dtype=torch.int)

# with torch.no_grad():
#     out = model(x)
# out

In [16]:

# # Generate text
# tokenizer = tiktoken.get_encoding('gpt2')
# def genarate_text(input, max_len, context_len):
#     model.eval()
#     for _ in range(max_len):
#         input = input[:, : context_len]
        
#         with torch.no_grad():
            
#             logits = model(input)
#             last_token_logits = logits[:, -1, :]
#             last_token_probs = torch.softmax(last_token_logits, dim=-1)        
#             top_prob_token = torch.argmax(last_token_probs, dim=-1, keepdim=True)
#             input = torch.cat([input, top_prob_token], dim=-1)
    
#     return input

# input = 'my name is'
# input = tokenizer.encode(input)
# input = torch.tensor(input).unsqueeze(0)
# print(input[:, :10])
# print(f'initial input : {input}')
# out = genarate_text(input, max_len=5, context_len=1024)
# print('out token', out)
# tokenizer.decode(out.squeeze().tolist())

In [17]:
with open('theverdict.txt','r') as f:
    text = f.read()

In [18]:
dataloader = create_dataloader(text, batch_size=4, num_workers=0)

In [19]:
for batch, target in dataloader:
    break

/Users/abdulvajid/AI/PROJECTS/LLM-FROM-SCRATCH/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [20]:
model = GPTModel(config)
input = torch.tensor(tokenizer.encode('hai')).unsqueeze(0)
model(input)

tensor([[[-4143.0698, -5602.8335, -3983.6526,  ..., -1917.2537,
          -8390.3008,  1517.2823]]], grad_fn=<ViewBackward0>)

In [ ]:
# Training
def train(model, train_dataloader:DataLoader, val_dataloader, optimizer:AdamW, epochs, val_freq:int, num_iter:int, device_man:str):
    if device_man:
        model.to(device_man)
    else:
        model.to(device)
        
    global_step = 1
    for epoch in range(epochs):
        model.train()
        train_loss_history = []
        val_loss_history = []
        
        for batch, target in train_dataloader:
            batch, target = batch.to(device), target.to(device)
            loss = calculate_batch_loss(model, batch, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            global_step+=1
            
            # validation
            if global_step % val_freq == 0:
                train_loss, val_loss = evaluate_model(model, train_dataloader, val_dataloader, num_iter)
                print(f"epoch: {epoch}: train_loss: {train_loss:.3f}, val_loss:{val_loss:.3f}")
                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)
                generate_sample_text(model)
            
 
def calculate_batch_loss(model, batch, target):
    logits = model(batch).flatten(0, 1)
    target = target.flatten()
    loss = torch.nn.functional.cross_entropy(logits, target)
    return loss

def evaluate_model(model, train_dataloader, val_dataloader, num_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calculate_dataloader_loss(model, train_dataloader, num_iter=num_iter)
        val_loss = calculate_dataloader_loss(model, val_dataloader, num_iter=num_iter)
    model.train()
    return train_loss, val_loss
               
def calculate_dataloader_loss(model, dataloader, num_iter):
    model.eval()
    total_loss = 0
    for i, (batch, target) in enumerate(dataloader):
        batch, target = batch.to(device), target.to(device)
        out = model(batch)
        loss = torch.nn.functional.cross_entropy(out.flatten(0,1), target=target.flatten())
        total_loss += loss.item() 
        if i+1 == num_iter: # i+1 since i start from 0
            break
    avg_loss = total_loss/num_iter
    return avg_loss

def generate_sample_text(model, starting_context:str='i am a good', tokenizer=tokenizer, max_len=10):
    model.eval()
    input_ids = tokenizer.encode(starting_context)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    input_ids = input_ids.to(device)
    for i in range(max_len):
        with torch.no_grad():
            logits = model(input_ids)
            last_token_logit = logits[:,-1,:]
            last_token = torch.argmax(last_token_logit, dim=-1).unsqueeze(0)
            input_ids = torch.cat([input_ids, last_token], dim=-1)
    print(tokenizer.decode(input_ids.squeeze().tolist()))

In [22]:
train_data = tokenizer.encode(text)[:int(0.8 * len(tokenizer.encode(text)))]
val_data = tokenizer.encode(text)[int(0.8 * len(tokenizer.encode(text))):]
train_text = tokenizer.decode(train_data)
val_text = tokenizer.decode(val_data)

In [23]:
model = GPTModel(config)
model.to(device);
train_dataloader = create_dataloader(txt=train_text, batch_size=4, max_len=256, stride=256)
val_dataloader = create_dataloader(txt=val_text, batch_size=2, max_len=256, stride=256)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=0.1)

In [24]:
train(model=model,
      train_dataloader=train_dataloader,
      val_dataloader=val_dataloader,
      optimizer=optimizer,
      epochs=10,
      val_freq=2,
      num_iter=5,
      device_man=None)

/Users/abdulvajid/AI/PROJECTS/LLM-FROM-SCRATCH/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch: 0: train_loss: 22374.307, val_loss:11665.787
i am a good-- the the the the the the the the the
epoch: 0: train_loss: 28182.380, val_loss:13801.577
i am a good savinding his Transcript 1970 was trials guts� Southeast
epoch: 1: train_loss: 33969.614, val_loss:17028.412
i am a goodHandlebr guts's quoteHandle Deb IDE Luo meticulous
epoch: 1: train_loss: 38334.548, val_loss:20072.380
i am a good I Ipod was have I I I I I
epoch: 2: train_loss: 73627.131, val_loss:39723.606
i am a good










epoch: 2: train_loss: 124782.834, val_loss:64841.919
i am a good of of of of of of of of of of
epoch: 3: train_loss: 107786.259, val_loss:54898.503
i am a good painting painting painting painting painting painting painting painting painting painting
epoch: 3: train_loss: 221407.534, val_loss:108948.791
i am a good without without without without without without without without myself without
epoch: 4: train_loss: 266935.700, val_loss:135701.038
i am a good one one one one one one one one one--
